In [1]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 29.7 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=ca3d255ff127c632049ce390e6d91e5dfb53230635c883ea2dc56304761d2495
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


In [3]:
!pip install beautifulsoup4

## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [6]:
%cd drive
!ls

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping
datasets_updated
scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
scrape_wikipedia_arabic_Alexander_Haig.ipynb
scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb


In [7]:
%cd MyDrive

[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [8]:
!ls

datasets_updated
scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
scrape_wikipedia_arabic_Alexander_Haig.ipynb
scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb


In [9]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [10]:
page = wptools.page('Israeli–Lebanese_conflict')
page.get_parse()

en.wikipedia.org (parse) Israeli–Lebanese_conflict
en.wikipedia.org (imageinfo) File:Israel and Lebanon.JPG
Israeli–Lebanese conflict (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Israel an...
  infobox: <dict(14)> conflict, partof, image, image_size, caption...
  iwlinks: <list(1)> https://en.wikisource.org/wiki/Cablegram_from...
  pageid: 6020082
  parsetree: <str(122272)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Israeli–Lebanese conflict
  wikibase: Q3241199
  wikidata_url: https://www.wikidata.org/wiki/Q3241199
  wikitext: <str(100914)> {{Short description|Conflict involving I...
}


In [11]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Israeli–Lebanese_conflict",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Israeli Lebanese conflict')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1936–39 Arab revolt in Palestine
1947 UN Partition Plan
1948 Arab–Israeli War
1948 Palestine war
1948 Palestinian exodus
1948 Palestinian expulsion and flight
1949 Armistice Agreements
1958 Lebanon Crisis
1968 Israeli raid on Lebanon
1971 JVP insurrection
1978 South Lebanon conflict
1982 Lebanon War
1982 Lebanon war
1982 kidnapping of Iranian diplomats
1987–1989 JVP insurrection
1990 Osh clashes
1991 Iraqi missile attacks against Israel
1992 Buenos Aires Israeli embassy bombing
1994 North Korean nuclear crisis
1996 shelling of Qana
1999 East Timorese crisis
1999 arrest of Iranian Jews
2000 Camp David Summit
2000 Hezbollah cross-border raid
2000–2006 Shebaa Farms conflict
2002 Arab League summit
2005 Hezbollah cross-border raid
2006 East Timorese crisis
2006 Gaza–Israel conflict
2006 Israeli operation in Beit Hanoun
2006 Lebanon War
2006 Lebanon war
2007 Lebanon conflict
2008 Israel–Hezbollah prisoner exchange
2008 Lebanon conflict
2008 Tibetan unrest
2008 conflict in Lebanon
200

In [12]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [13]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [14]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [15]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1936–39 Arab revolt in Palestine
en.wikipedia.org (imageinfo) File:Train hostages.jpg
en.wikipedia.org (parse) 1947 UN Partition Plan
en.wikipedia.org (imageinfo) File:UN Palestine Partition Versions...
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1948 Palestinian expulsion and flight
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1958 Lebanon Crisis
en.wikipedia.org (imageinfo) File:Foxhole - Lebanon - Beirut - Ju...
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1971 JVP insurrection
en.wikipedia.org (imageinfo) File:Deniyau.png
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 Lebanon war
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
e

['Al-Aqsa Martyrs Brigades']
-----------------
name:  ['Al-Aqsa Martyrs Brigades']
founders:  
ideology:  ['<!-- Palestinian nationalism and Anti-Zionism can be assumed', 'but DONT add any other ideologies unless you have a RECENT erence. -->', 'Plainlist', 'Anti-Zionism', 'Secularism', 'Palestinian nationalism']
position:  -
START
Cannot find the link: http://en.wikipedia.org/wiki/AlAqsa Martyrs Brigades
name:  Al-Aqsa Martyrs Brigades
name ar:  كتائب شهداء الأقصى (مترجمه)
name ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Palestinian nationalism and AntiZionism can be assumed
ideology:  <!-- Palestinian nationalism and Anti-Zionism can be assumed
ideology ar:  يمكن افتراض القومية الفلسطينية ومعاداة الصهيونية (مترجمه)
ideo ar link:  -1
Cannot find the link: http://en.wikipedia.org/wiki/but DONT add any other ideologies unless you have a RECENT erence
ideology:  but DONT add any other ideologies unless you have a RECENT erence. -->
ideology ar:  لكن لا تضيف 

en.wikipedia.org (parse) Al-Jadeed
en.wikipedia.org (parse) Al-Manar
en.wikipedia.org (parse) Al-Mourabitoun


[]
-----------------
name:  []
founders:  
ideology:  ['Pan-Arabism', 'Anti-imperialism', 'Nasserism', 'Arab socialism', 'Arab nationalism']
position:  ['Left-wing']
START
END
START
Cannot find the link: http://en.wikipedia.org/wiki/PanArabism
ideology:  Pan-Arabism
ideology ar:  القومية العربية (مترجمه)
ideo ar link:  -1
ideology:  Anti-imperialism
ideology ar:  معاداة الاستعمارية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%A7%D8%B3%D8%AA%D8%B9%D9%85%D8%A7%D8%B1%D9%8A%D8%A9
ideology:  Nasserism
ideology ar:  التيار الناصري 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AA%D9%8A%D8%A7%D8%B1_%D8%A7%D9%84%D9%86%D8%A7%D8%B5%D8%B1%D9%8A
ideology:  Arab socialism
ideology ar:  اشتراكية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Arab nationalism
ideology ar:  قومية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D

en.wikipedia.org (parse) Al-Qaeda insurgency in Yemen
en.wikipedia.org (imageinfo) File:Yemeni Civil War.svg
en.wikipedia.org (parse) Al Jazeera English
en.wikipedia.org (parse) Al Jazeera Media Network
en.wikipedia.org (parse) Al Manar
en.wikipedia.org (parse) Al Mayadeen
en.wikipedia.org (parse) Alarabiyah
en.wikipedia.org (parse) All-Palestine Government
en.wikipedia.org (parse) Amal Movement


...

en.wikipedia.org (parse) South Lebanon conflict (1985–2000)
en.wikipedia.org (imageinfo) File:South Lebanon conflict montage.png
en.wikipedia.org (parse) South Lebanon security belt
en.wikipedia.org (parse) South Thailand insurgency
en.wikipedia.org (imageinfo) File:Souththailandmap.svg
en.wikipedia.org (parse) South Yemen insurgency
en.wikipedia.org (imageinfo) File:Yemeni Civil War.svg
en.wikipedia.org (parse) Soviet Union
en.wikipedia.org (parse) Spillover of the Syrian Civil War
en.wikipedia.org (imageinfo) File:Syrian, Iraqi, and Lebanese ins...
en.wikipedia.org (parse) Sri Lankan Civil War
en.wikipedia.org (imageinfo) File:Sri Lanka North Eastern Provinc...
en.wikipedia.org (parse) Stuxnet
en.wikipedia.org (parse) Suez Crisis
en.wikipedia.org (imageinfo) File:Tanks Destroyed Sinai.jpg
en.wikipedia.org (parse) Surveillance aircraft
en.wikipedia.org (parse) Sykes–Picot Agreement
en.wikipedia.org (imageinfo) File:MPK1-426 Sykes Picot Agreement ...
en.wikipedia.org (parse) Syria
en.w

Rabie Banat
[[Antoun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabie Banat,Antoun Saadeh,
ideology:  ['Syrian nationalism', 'Social nationalism', 'Economic populism', 'collapsible list', 'Antisemitism', 'Syrian irredentism', 'ubl', 'Anti-communism', 'Secularism', 'Fascism']
position:  ['Syncretic efn', 'In the past', 'the party or elements of its ideology or membership have been erred to as belonging to both the political left', 'sometimes being labeled far-right.', 'and right', 'sometimes being labeled far-right.', 'and right']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Syrian nationalism
ideology ar:  القومية السورية 
ideo ar link:  https://ar.wi

en.wikipedia.org (parse) Syrian civil war
en.wikipedia.org (parse) Taba Summit
en.wikipedia.org (parse) Tajikistani Civil War
en.wikipedia.org (imageinfo) File:Spetsnaz troopers during the 19...
en.wikipedia.org (parse) Tamar gas field
en.wikipedia.org (parse) Tehran (TV series)
en.wikipedia.org (imageinfo) File:Tehran TV poster.jpg
en.wikipedia.org (parse) The CIA World Factbook
en.wikipedia.org (imageinfo) File:The World Factbook emblema.svg
en.wikipedia.org (parse) The Clinton Parameters
en.wikipedia.org (parse) The Daily Telegraph
en.wikipedia.org (imageinfo) File:The Daily Telegraph (British ne...
en.wikipedia.org (parse) The Jerusalem Post
en.wikipedia.org (parse) The National (Abu Dhabi)
en.wikipedia.org (imageinfo) File:The National 27 June 2018 Front...
en.wikipedia.org (parse) The Times of Israel
en.wikipedia.org (imageinfo) File:Times of Israel office, Jerusalem.JPG
en.wikipedia.org (parse) The Washington Post
en.wikipedia.org (imageinfo) File:The-Washington-Post-10-June-202

skipping Wikipedia:Citation needed
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox Israel–Lebanon conflict
skipping Template:Cite book
skipping Template:Cite journal
skipping Template:Cite web
skipping Template:Country data Iran
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data PLO
skipping Template:Iran–Israel proxy conflict
skipping Template:Israeli wars
skipping Template:Post-Cold War Asian conflicts
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox Israel–Lebanon conflict
skipping Template talk:Iran–Israel proxy co

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [16]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Al-Aqsa Martyrs Brigades,<!-- Palestinian nationalism and Anti-Zionism ...,-,
1,,"Pan-Arabism,Anti-imperialism,Nasserism,Arab so...",Left-wing,
2,Amal Movement,"Traditionalist conservatism,ubl,Anti-Zionism,L...",Centre-right,"Nabih Berri,Musa al-Sadr,Hussein el-Husseini,M..."
3,Amal Movement,"Traditionalist conservatism,ubl,Anti-Zionism,L...",Centre-right,"Nabih Berri,Musa al-Sadr,Hussein el-Husseini,M..."
4,Arab Higher Committee,"Palestinian nationalism,Arab nationalism,Anti-...",-,
5,Arab Liberation Army,"Anti-Zionism,Arab nationalism,Pan-Arabism,Anti...",-,
6,Arab Liberation Front,Baathism,-,
7,Army of Islam,"Jihadism,Salafism,Sunni Islamism",-,
8,Army of the Holy War,"Pan-Arabism,Islamism,Anti-Zionism,Palestinian ...",-,
9,As-Saiqa,"Neo-Baathism,Palestinian nationalism,Pan-Arabism",Far-left,


In [17]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,كتائب شهداء الأقصى (مترجمه),يمكن افتراض القومية الفلسطينية ومعاداة الصهيون...,-,
1,,"القومية العربية (مترجمه),معاداة الاستعمارية ,ا...",يسارية,
2,حركة أمل,"تقليدوية ,أوبل (مترجمه),معاداة الصهيونية ,قومي...",يمين الوسط (مترجمه),"نبيه بري ,موسى الصدر (مترجمه),حسين الحسيني (مت..."
3,حركة أمل,"تقليدوية ,أوبل (مترجمه),معاداة الصهيونية ,قومي...",يمين الوسط (مترجمه),"نبيه بري ,موسى الصدر (مترجمه),حسين الحسيني (مت..."
4,اللجنة العربية العليا,"وطنية فلسطينية ,قومية عربية ,معاداة الصهيونية",-,
5,جيش الإنقاذ,"معاداة الصهيونية ,قومية عربية ,القومية العربية...",-,
6,جبهة التحرير العربية,بعثية,-,
7,جيش الاسلام (مترجمه),"الجهادية (مترجمه),السلفية ,إسلاموية",-,
8,جيش الجهاد المقدس,"القومية العربية (مترجمه),إسلاموية ,معاداة الصه...",-,
9,كما الصاعقة (مترجمه),"البعثية الجديدة (مترجمه),وطنية فلسطينية ,القوم...",أقصى اليسار (مترجمه),


In [18]:
datasets_dir = 'datasets_updated/Israeli–Lebanese_conflict/'

In [19]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [20]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [21]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1936–39 Arab revolt in Palestine
en.wikipedia.org (imageinfo) File:Train hostages.jpg
en.wikipedia.org (parse) 1947 UN Partition Plan
en.wikipedia.org (imageinfo) File:UN Palestine Partition Versions...
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1948 Palestinian expulsion and flight
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1958 Lebanon Crisis
en.wikipedia.org (imageinfo) File:Foxhole - Lebanon - Beirut - Ju...
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1971 JVP insurrection
en.wikipedia.org (imageinfo) File:Deniyau.png
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 Lebanon war
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
e

['Abu Nidal']
name_inserted before: ['Abu Nidal']
name_inserted after: Abu Nidal
-----------------
name:  ['Abu Nidal']
native_name:  ['أبو نضال', 'lang', 'ar']
nickname:  -
religion:  -
party:  []
nationality:  ['Palestinian']
name ar:  صبري البنا 
START
END
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  صبري البنا 
religion ar:  []
party ar:  []
nationality ar:  ['فلسطينيون ']


en.wikipedia.org (parse) Afghan Civil War (1989–1992)
en.wikipedia.org (imageinfo) File:Political map of Afghanistan in...
en.wikipedia.org (parse) Afghan Civil War (1992–1996)
en.wikipedia.org (parse) Afghan Civil War (1996–2001)
en.wikipedia.org (imageinfo) File:Afghanistan politisch 2000.png
en.wikipedia.org (parse) Afghan conflict
en.wikipedia.org (parse) Afghanistan–Pakistan border skirmishes
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Ahmad Reza Djalali


['Ahmad Reza Djalali']
name_inserted before: ['Ahmad Reza Djalali']
name_inserted after: Ahmad Reza Djalali
-----------------
name:  ['Ahmad Reza Djalali']
native_name:  ['lang', 'fa', 'احمدرضا جلالی']
nickname:  -
religion:  -
party:  -
nationality:  ['Iranian-Swedish']
name ar:  أحمد رضا جلالي (مترجمه)
START
Cannot find the link: http://en.wikipedia.org/wiki/IranianSwedish
nationality:  Iranian-Swedish
nationality ar:  إيراني سويدي (مترجمه)
nationality ar link:  -1
END
name ar:  أحمد رضا جلالي (مترجمه)
religion ar:  -
party ar:  -
nationality ar:  ['إيراني سويدي (مترجمه)']


en.wikipedia.org (parse) Ahmed Shukeri
en.wikipedia.org (imageinfo) File:Ahmad Shukeiri cropped.jpg
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades
en.wikipedia.org (imageinfo) File:Al Aqsa Martyrs' Brigades Flag.gif
en.wikipedia.org (parse) Al-Jadeed
en.wikipedia.org (parse) Al-Manar
en.wikipedia.org (parse) Al-Mourabitoun
en.wikipedia.org (parse) Al-Qaeda insurgency in Yemen
en.wikipedia.org (imageinfo) File:Yemeni Civil War.svg
en.wikipedia.org (parse) Al Jazeera English
en.wikipedia.org (parse) Al Jazeera Media Network
en.wikipedia.org (parse) Al Manar
en.wikipedia.org (parse) Al Mayadeen
en.wikipedia.org (parse) Alarabiyah
en.wikipedia.org (parse) All-Palestine Government
en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) Amal movement
en.wikipedia.org (parse) American trusteeship proposal for Palestine
en.wikipedia.org (parse) Andijan massacre
en.wikipedia.org (i

...

en.wikipedia.org (parse) Yassir Arafat
en.wikipedia.org (imageinfo) File:Leader of the PLO, Yasser Arafa...


['Yasser Arafat']
name_inserted before: ['Yasser Arafat']
name_inserted after: Yasser Arafat
-----------------
name:  ['Yasser Arafat']
native_name:  ['nobold', 'ياسر عرفات']
nickname:  ['Abu Ammar']
religion:  -
party:  ['Fatah']
nationality:  -
Cannot translate phrase `Yassir Arafat` to arabic
name ar:  -1
START
political party:  Fatah
political party ar:  حركة فتح 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D9%81%D8%AA%D8%AD
END
name ar:  -1
religion ar:  ['حركة فتح ']
party ar:  ['Fatah']
nationality ar:  -


en.wikipedia.org (parse) Yatir Forest
en.wikipedia.org (parse) Yemen
en.wikipedia.org (parse) Yemeni civil war (1994)
en.wikipedia.org (imageinfo) File:Yemen 1994 civil-war 03.jpg
en.wikipedia.org (parse) Yemeni civil war (2014–present)
en.wikipedia.org (imageinfo) File:Yemeni Civil War.svg
en.wikipedia.org (parse) Yemeni crisis
en.wikipedia.org (parse) Ynet
en.wikipedia.org (parse) Yom Kippur War
en.wikipedia.org (imageinfo) File:Yom Kippur War Montage.png
en.wikipedia.org (parse) Zar'it
en.wikipedia.org (parse) Zar'it-Shtula incident
en.wikipedia.org (imageinfo) File:Zarit incident map.png
en.wikipedia.org (parse) Zeev Maoz
en.wikipedia.org (parse) Zeina Karam
en.wikipedia.org (parse) File:Middle east graphic 2003.jpg


skipping Wikipedia:Citation needed
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox Israel–Lebanon conflict
skipping Template:Cite book
skipping Template:Cite journal
skipping Template:Cite web
skipping Template:Country data Iran
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data PLO
skipping Template:Iran–Israel proxy conflict
skipping Template:Israeli wars
skipping Template:Post-Cold War Asian conflicts
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox Israel–Lebanon conflict
skipping Template talk:Iran–Israel proxy co

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [22]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Abu Nidal,,Palestinian,-
1,Ahmad Reza Djalali,-,Iranian-Swedish,-
2,Ardeshir Hosseinpour,-,-,-
3,Bachir Gemayel,Kataeb Party,-,-
4,nobold,Likud,-,-
5,Elhanan Tannenbaum,-,-,-
6,Fereydoon Abbasi Davani,Association of Islamic Revolution Loyalists,Iranian,-
7,Fouad Siniora,Future Movement,-,-
8,Fouad Siniora,Future Movement,-,-
9,Hussein bin Talal,-,-,Sunni Islam


In [23]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,فلسطينيون,,صبري البنا
1,-,إيراني سويدي (مترجمه),-,أحمد رضا جلالي (مترجمه)
2,-,-,-,أردشير حسين بور (مترجمه)
3,-,-,حزب الكتائب اللبنانية,بشير الجميل
4,-,-,ليكود,بنيامين نتنياهو
5,-,-,-,إلحانان تاننباوم
6,-,إيراني (مترجمه),جمعية الموالين للثورة الإسلامية,فريدون عباسي دوائي
7,-,-,تيار المستقبل,فؤاد السنيورة
8,-,-,تيار المستقبل,فؤاد السنيورة
9,أهل السنة والجماعة,-,-,الحسين بن طلال


In [24]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [25]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [26]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1936–39 Arab revolt in Palestine
en.wikipedia.org (imageinfo) File:Train hostages.jpg
en.wikipedia.org (parse) 1947 UN Partition Plan
en.wikipedia.org (imageinfo) File:UN Palestine Partition Versions...
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1948 Palestinian expulsion and flight
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1958 Lebanon Crisis
en.wikipedia.org (imageinfo) File:Foxhole - Lebanon - Beirut - Ju...
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1971 JVP insurrection
en.wikipedia.org (imageinfo) File:Deniyau.png
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 Lebanon war
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
e

-----------------
name:  Aadloun
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'Flag']
name ar:  عدلون 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
END


en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (imageinfo) File:President Trump and The First L...
en.wikipedia.org (parse) Abu Nidal
en.wikipedia.org (imageinfo) File:Abu Nidal.gif
en.wikipedia.org (parse) Afghan Civil War (1989–1992)
en.wikipedia.org (imageinfo) File:Political map of Afghanistan in...
en.wikipedia.org (parse) Afghan Civil War (1992–1996)
en.wikipedia.org (parse) Afghan Civil War (1996–2001)
en.wikipedia.org (imageinfo) File:Afghanistan politisch 2000.png
en.wikipedia.org (parse) Afghan conflict
en.wikipedia.org (parse) Afghanistan–Pakistan border skirmishes
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Ahmad Reza Djalali
en.wikipedia.org (parse) Ahmed Shukeri
en.wikipedia.org (imageinfo) File:Ahmad Shukeiri cropped.jpg
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades
en.wikipedia.org (imageinfo) File:Al Aq

-----------------
name:  Anjar
subdivision_type:  ['Country']
subdivision_name:  ['LBN']
name ar:  عنجر (مترجمه)
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  LBN
subdivision_name ar:  لبنان (مترجمه)
subdivision_name ar link:  -1
END


en.wikipedia.org (parse) Annapolis Conference
en.wikipedia.org (parse) Anti-Semitism
en.wikipedia.org (parse) April 2015 Qalamoun incident
en.wikipedia.org (imageinfo) File:Battle of Qalamoun.svg
en.wikipedia.org (parse) April 2024 Iranian strikes against Israel
en.wikipedia.org (parse) Arab
en.wikipedia.org (parse) Arab Higher Committee
en.wikipedia.org (parse) Arab League
en.wikipedia.org (parse) Arab Liberation Army
en.wikipedia.org (parse) Arab Liberation Front
en.wikipedia.org (parse) Arab Spring
en.wikipedia.org (parse) Arab Winter
en.wikipedia.org (imageinfo) File:Char Etat islamique Raqqa.jpg
en.wikipedia.org (parse) Arab separatism in Khuzestan
en.wikipedia.org (imageinfo) File:Locator map Iran Khuzestan Prov...
en.wikipedia.org (parse) Arab–Israeli alliance
en.wikipedia.org (parse) Arab–Israeli conflict
en.wikipedia.org (parse) Arab–Israeli normalization
en.wikipedia.org (parse) Ardeshir Hosseinpour
en.wikipedia.org (parse) Army of Islam (Gaza)
en.wikipedia.org (parse) Army o

...

en.wikipedia.org (parse) Unmanned Aerial Vehicle
en.wikipedia.org (parse) Victoria Affair
en.wikipedia.org (parse) Views on military action against Iran
en.wikipedia.org (parse) Walid Jumblatt
en.wikipedia.org (imageinfo) File:Walid Jumblatt 6C2.jpg
en.wikipedia.org (parse) Walla!
en.wikipedia.org (parse) War in Afghanistan (2001–2021)
en.wikipedia.org (parse) War in Iraq (2013–2017)
en.wikipedia.org (imageinfo) File:ISOF APC on the street of Mosul...
en.wikipedia.org (parse) War of Attrition
en.wikipedia.org (parse) War on terror
en.wikipedia.org (parse) War over Water (Jordan River)
en.wikipedia.org (parse) War over Water (Jordan river)
en.wikipedia.org (parse) Wayback Machine
en.wikipedia.org (parse) Western Iran clashes (2016–present)
en.wikipedia.org (parse) White Paper of 1939
en.wikipedia.org (imageinfo) File:1939 White Paper cmd 6019.djvu
en.wikipedia.org (parse) Wolf Armoured Vehicle
en.wikipedia.org (imageinfo) File:Zeev-jeep002.jpg
en.wikipedia.org (parse) World War I
en.wik

-----------------
name:  Yemen
subdivision_type:  -
subdivision_name:  -
name ar:  اليمن 


en.wikipedia.org (parse) Yemeni civil war (1994)
en.wikipedia.org (imageinfo) File:Yemen 1994 civil-war 03.jpg
en.wikipedia.org (parse) Yemeni civil war (2014–present)
en.wikipedia.org (imageinfo) File:Yemeni Civil War.svg
en.wikipedia.org (parse) Yemeni crisis
en.wikipedia.org (parse) Ynet
en.wikipedia.org (parse) Yom Kippur War
en.wikipedia.org (imageinfo) File:Yom Kippur War Montage.png
en.wikipedia.org (parse) Zar'it
en.wikipedia.org (parse) Zar'it-Shtula incident
en.wikipedia.org (imageinfo) File:Zarit incident map.png
en.wikipedia.org (parse) Zeev Maoz
en.wikipedia.org (parse) Zeina Karam
en.wikipedia.org (parse) File:Middle east graphic 2003.jpg


skipping Wikipedia:Citation needed
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox Israel–Lebanon conflict
skipping Template:Cite book
skipping Template:Cite journal
skipping Template:Cite web
skipping Template:Country data Iran
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data PLO
skipping Template:Iran–Israel proxy conflict
skipping Template:Israeli wars
skipping Template:Post-Cold War Asian conflicts
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox Israel–Lebanon conflict
skipping Template talk:Iran–Israel proxy co

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [27]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Aadloun,Country,"Lebanon,Flag"
1,Anjar,Country,LBN
2,Iraq,-,-
3,Baalbek,Country,"Lebanon,Flag"
4,Beirut,Country,Lebanon
5,Dahieh,Country,Lebanon
6,-,Country,Syria
7,Egypt,-,-
8,European Union,-,-
9,France,-,-


In [28]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,"لبنان ,علم (راية)",بلد,عدلون
1,لبنان (مترجمه),بلد,عنجر (مترجمه)
2,-,-,العراق
3,"لبنان ,علم (راية)",بلد,بعلبك
4,لبنان,بلد,بيروت
5,لبنان,بلد,الضاحية الجنوبية (بيروت)
6,-,-,مصر
7,-,-,الاتحاد الأوروبي
8,-,-,فرنسا
9,"انظر وضع مرتفعات الجولان (مترجمه),الأراضي السو...",حالة (مترجمه),هضبة الجولان


In [29]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')